In [1]:
import gensim.downloader as api
wv = api.load('glove-wiki-gigaword-300')

In [2]:
import numpy as np
import pandas as pd
from utils.dataset import DataSet
from utils.generate_test_splits import generate_hold_out_split, read_ids

In [3]:
d = DataSet()
generate_hold_out_split(d)
trainID = set(read_ids("training_ids.txt", "splits"))
valID = set(read_ids("hold_out_ids.txt", "splits"))

Reading dataset
Total stances: 49972
Total bodies: 1683


# Hyperparams

In [4]:
MAX_SENT_PER_ART = 5
MAX_SENT_LEN = 20
MAX_VOCAB = 50000
VECTOR_SIZE = 300

# Set up training and validation data

In [5]:
train_stances = [stance for stance in d.stances if stance['Body ID'] in trainID]
train_headlines = [stance['Headline'] for stance in train_stances]
train_labels = [stance['Stance'] for stance in train_stances]
train_body = [d.articles[stance['Body ID']] for stance in train_stances]

val_stances = [stance for stance in d.stances if stance['Body ID'] in valID]
val_headlines = [stance['Headline'] for stance in val_stances]
val_labels = [stance['Stance'] for stance in val_stances]
val_body = [d.articles[stance['Body ID']] for stance in val_stances]

# Vectorization and Tokenization

In [6]:
import nltk
from nltk import tokenize
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.layers import TextVectorization
nltk.download('punkt')

vectorizer = TextVectorization(max_tokens=MAX_VOCAB, output_sequence_length=MAX_SENT_LEN)
vectorizer.adapt(train_body + train_headlines + val_body + val_headlines)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

[nltk_data] Downloading package punkt to /home/sw26wong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
sent_tok_art = []
for article in train_body:
    sent_tok_art.append(tokenize.sent_tokenize(article))

vsent_tok_art = []
for article in val_body:
    vsent_tok_art.append(tokenize.sent_tokenize(article))

In [8]:
X_train_body = np.zeros((len(train_stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')

for i, article in enumerate(sent_tok_art):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_train_body[i][j][k] = word_index.get(word, 1) # get else UNK

X_train_head = np.zeros((len(train_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(train_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_train_head[i][j] = word_index.get(word, 1)

X_val_body = np.zeros((len(val_stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')

for i, article in enumerate(vsent_tok_art):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_val_body[i][j][k] = word_index.get(word, 1)

X_val_head = np.zeros((len(val_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(val_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_val_head[i][j] = word_index.get(word, 1)

In [9]:
targets = pd.Series(train_labels)
one_hot = pd.get_dummies(targets,sparse = True)
one_hot_labels = np.asarray(one_hot)
y_train = one_hot_labels

In [10]:
targets = pd.Series(val_labels)
one_hot = pd.get_dummies(targets,sparse = True)
one_hot_labels = np.asarray(one_hot)
y_val = one_hot_labels

In [11]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']

# Create Embedding Matrix from Google word2vec

In [12]:
vocab_size = len(word_index)
embedding_matrix = np.zeros((vocab_size+1, VECTOR_SIZE))

for word, i in word_index.items():
    try:
        v = wv[word]
        embedding_matrix[i] = v
    except KeyError:
        pass

In [13]:
import gc
del wv
gc.collect()

88

# Model

In [14]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, TimeDistributed, Activation
from keras.layers import Flatten, Permute, merge, Input
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input,Dense,multiply,concatenate,Dropout
from keras.layers import GRU, Bidirectional

In [20]:
hidden_size = VECTOR_SIZE

sentence_input = Input(shape=(MAX_SENT_LEN,),dtype='int32')
embedded_sequences = Embedding(output_dim=hidden_size, 
                               input_dim=vocab_size+1, 
                               input_length=MAX_SENT_LEN,
                               weights=[embedding_matrix],
                               trainable=True,
                               mask_zero=False,)(sentence_input)
l_LSTM = Bidirectional(LSTM(hidden_size, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(hidden_size))(l_LSTM)
l_dense = Flatten()(l_dense)
sentEncoder = Model(sentence_input,l_dense)

body_input = Input(shape=(MAX_SENT_PER_ART,MAX_SENT_LEN,),dtype = 'int32')
body_encoder = TimeDistributed(sentEncoder)(body_input)

l_LSTM_sent = Bidirectional(LSTM(hidden_size,return_sequences=True))(body_encoder)
l_dense_sent = TimeDistributed(Dense(hidden_size))(l_LSTM_sent)
l_dense_sent = Flatten()(l_dense_sent)

heading_input = Input(shape=(MAX_SENT_LEN,) ,dtype = 'int32')
heading_embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size+1, \
                                       input_length=(MAX_SENT_LEN,), \
                                      weights=[embedding_matrix],
                                      trainable=True,
                                      mask_zero=False,)(heading_input)
h_dense = Dense(hidden_size,activation='relu')(heading_embedded_sequences)
h_flatten = Flatten()(h_dense)

article_output = concatenate([l_dense_sent,h_flatten],name = 'concatenate_heading')

news_vestor = Dense(hidden_size,activation = 'relu')(article_output)
preds = Dense(4 ,activation='softmax')(news_vestor)
model = Model([body_input,heading_input],[preds])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

model.fit([X_train_body,X_train_head],[y_train], validation_data=([X_val_body,X_val_head],[y_val]), epochs=25 , batch_size=128)

Epoch 1/25
316/316 [==============================] - 210s 644ms/step - loss: 0.5860 - categorical_accuracy: 0.7871 - val_loss: 0.6747 - val_categorical_accuracy: 0.7146
Epoch 2/25
316/316 [==============================] - 207s 657ms/step - loss: 0.2630 - categorical_accuracy: 0.8961 - val_loss: 0.5735 - val_categorical_accuracy: 0.7775
Epoch 3/25
316/316 [==============================] - 162s 512ms/step - loss: 0.1527 - categorical_accuracy: 0.9405 - val_loss: 0.7002 - val_categorical_accuracy: 0.7905
Epoch 4/25
316/316 [==============================] - 149s 472ms/step - loss: 0.1032 - categorical_accuracy: 0.9611 - val_loss: 0.7918 - val_categorical_accuracy: 0.7716
Epoch 5/25
316/316 [==============================] - 155s 490ms/step - loss: 0.0760 - categorical_accuracy: 0.9726 - val_loss: 0.8814 - val_categorical_accuracy: 0.7910
Epoch 6/25
316/316 [==============================] - 153s 485ms/step - loss: 0.0587 - categorical_accuracy: 0.9785 - val_loss: 0.8437 - val_categoric

In [21]:
cd = DataSet("competition_test")


test_stances = cd.stances
test_headlines = [stance['Headline'] for stance in test_stances]
test_labels = [stance['Stance'] for stance in test_stances]
test_body = [cd.articles[stance['Body ID']] for stance in test_stances]

X_test_body = np.zeros((len(cd.stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')
sent_tok_test = []
for article in test_body:
    sent_tok_test.append(tokenize.sent_tokenize(article))

for i, article in enumerate(sent_tok_test):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_test_body[i][j][k] = word_index.get(word, 1)

X_test_head = np.zeros((len(test_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(test_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_test_head[i][j] = word_index.get(word, 1)


predictions = model.predict([X_test_body,X_test_head])

predicted_label = [LABELS[max([0, 1, 2, 3], key=lambda x: p[x])] for p in predictions]

Reading dataset
Total stances: 25413
Total bodies: 904


In [22]:
score = sum([pl == a for pl, a in zip(predicted_label, test_labels)])/len(test_labels)
score

0.6924802266556487

# Glove 100
- 66% with masked zeros, not trainable

- 68% with no mask zero, not trainable

- 69% no mask zero, trainable

- 65% no mask zero, trainable + double-relu

- 66% no mask zero, trainable + single-relu

- 67.5% no mask zero, not trainable, 2048 batch
- 72.5% no mask zero, not trainable, 1024 batch
- 72.3% no mask zero, not trainable, 512 batch
- 72.0% no mask zero, not trainable 256 batch
- 72.8% no mask zero, not trainable, 128 batch

- 73.2% no mask zero, not trainable, 128 batch 25 epoch

- 71.9% no mask zero, not trainable, 1024 batch 30 epochs
- 72.2% no mask zero, not trainable, 1024 batch 15 epochs
- 73.22% no mask zero, not trainable, 1024 batch 20 epochs
- 72.9% no mask zero, not trainable, 1024 batch 25 epochs
- 72.7% no mask zero, not trainable, 1024 batch 30 epochs

# Glove 200
- 68% no mask zero, trainable + single-relu

# Glove 300
- 69% trainable 25 epoch
- 72% no mask zero, trainable
- 72.3% no mask zero, not trainable
- 73.8% no mask zero, not trainable, 128 batch 25 epoch

In [23]:
model.save("glove300-embeddings-trainable")

INFO:tensorflow:Assets written to: glove300-embeddings-trainable/assets


INFO:tensorflow:Assets written to: glove300-embeddings-trainable/assets
